0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input, Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, LeakyReLU, Dropout, Embedding, Concatenate
from matplotlib import pyplot as plt
from keras.layers import Conv2D, Conv2DTranspose, LeakyReLU, BatchNormalization
from keras.models import save_model
from math import sqrt
from numpy import asarray
from numpy.random import randn
from keras.models import load_model

##Loading and Preprocessing the dataset

In [ ]:
def load_fer2013_dataset():
    # Load the FER2013 dataset CSV file
    data = pd.read_csv('/content/fer_chnge.csv')

    # Extract pixel and emotion labels
    pixels = data['pixels'].values
    emotions = data['emotion'].values

    X = []
    y = []
    for pixel_sequence, emotion in zip(pixels, emotions):
        # Parse the pixel sequence string into a list of integers
        pixel_values = [int(pixel) for pixel in pixel_sequence.split()]

        # Ensure that the pixel values array matches the expected size (48x48 = 2304 pixels)
        if len(pixel_values) != 2304:
            continue  # Skip sequences that are not 48x48

        # Reshape the pixel values into a 48x48 grayscale image (1 channel)
        image_array = np.array(pixel_values, dtype=np.uint8).reshape(48, 48, 1)

        # Normalize pixel values to range [0, 1]
        image_array = image_array.astype('float32') / 255.0

        # Append the processed image array and corresponding emotion label to the lists
        X.append(image_array)
        y.append(emotion)

    # Convert the lists to numpy arrays
    X = np.array(X)
    y = np.array(y, dtype='int32')

    return (X, y)



##Discriminator Model

In [ ]:
def define_discriminator(in_shape=(48, 48, 1), n_classes=7):
    # Label input
    in_label = Input(shape=(1,))
    # Embedding for categorical input
    li = Embedding(n_classes, 50)(in_label)
    # Scale up to image dimensions with linear activation
    n_nodes = in_shape[0] * in_shape[1]
    li = Dense(n_nodes)(li)
    # Reshape to additional channel
    li = Reshape((in_shape[0], in_shape[1], 1))(li)
    # Image input
    in_image = Input(shape=in_shape)
    # Concatenate label as a channel
    merge = Concatenate()([in_image, li])
    # Downsample
    fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(merge)
    fe = LeakyReLU(alpha=0.2)(fe)
    # Downsample
    fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    # Flatten feature maps
    fe = Flatten()(fe)
    # Dropout
    fe = Dropout(0.4)(fe)
    # Output
    out_layer = Dense(1, activation='sigmoid')(fe)
    # Define model
    model = Model([in_image, in_label], out_layer)
    # Compile model
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model
d=define_discriminator()
d.summary()


Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_9 (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 embedding_4 (Embedding)     (None, 1, 50)                350       ['input_9[0][0]']             
                                                                                                  
 dense_8 (Dense)             (None, 1, 2304)              117504    ['embedding_4[0][0]']         
                                                                                                  
 input_10 (InputLayer)       [(None, 48, 48, 1)]          0         []                            
                                                                                            

##Generator Model

In [ ]:
def define_generator(latent_dim=100, n_classes=7):
    # Label input
    in_label = Input(shape=(1,))
    # Embedding for categorical input
    li = Embedding(n_classes, 50)(in_label)
    # Linear multiplication
    n_nodes = 12 * 12 * 128  # Calculate the correct number of nodes
    li = Dense(n_nodes)(li)
    li = LeakyReLU(alpha=0.2)(li)
    # Reshape to 12x12 feature maps
    li = Reshape((12, 12, 128))(li)
    # Image generator input
    in_lat = Input(shape=(latent_dim,))
    # Foundation for 12x12 image
    n_nodes = 128 * 12 * 12
    gen = Dense(n_nodes)(in_lat)
    gen = LeakyReLU(alpha=0.2)(gen)
    gen = Reshape((12, 12, 128))(gen)
    # Merge image generator and label input
    merge = Concatenate()([gen, li])
    # Upsample to 24x24
    gen = Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same')(merge)
    gen = LeakyReLU(alpha=0.2)(gen)
    # Upsample to 48x48
    gen = Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same')(gen)
    gen = LeakyReLU(alpha=0.2)(gen)
    # Output
    out_layer = Conv2D(1, (12, 12), activation='tanh', padding='same')(gen)
    # Define model
    model = Model([in_lat, in_label], out_layer)
    return model
g=define_generator()
g.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_11 (InputLayer)       [(None, 1)]                  0         []                            
                                                                                                  
 input_12 (InputLayer)       [(None, 100)]                0         []                            
                                                                                                  
 embedding_5 (Embedding)     (None, 1, 50)                350       ['input_11[0][0]']            
                                                                                                  
 dense_11 (Dense)            (None, 18432)                1861632   ['input_12[0][0]']            
                                                                                            

##GAN Model(D+G)

In [ ]:
def define_gan(g_model, d_model):
    # Make weights in the discriminator not trainable
    d_model.trainable = False
    # Get noise and label inputs from generator model
    gen_noise, gen_label = g_model.input
    # Get image output from the generator model
    gen_output = g_model.output
    # Connect image output and label input from generator as inputs to discriminator
    gan_output = d_model([gen_output, gen_label])
    # Define GAN model as taking noise and label and outputting a classification
    model = Model([gen_noise, gen_label], gan_output)
    # Compile model
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

In [ ]:
def generate_fake_samples(generator, latent_dim, n_samples, n_classes=7):
    # Generate points in latent space
    z_input = np.random.randn(latent_dim * n_samples)
    z_input = z_input.reshape(n_samples, latent_dim)
    # Generate random class labels
    labels = np.random.randint(0, n_classes, n_samples).reshape(-1, 1)
    # Generate images
    X_fake = generator.predict([z_input, labels])
    return X_fake, labels

##Train the GAN

In [ ]:
import os
save_dir = '/content/drive/MyDrive/cgan2'
os.makedirs(save_dir, exist_ok=True)

def train_gan(g_model, d_model, gan_model, dataset, latent_dim, n_epochs, n_batch):
    # Unpack dataset
    X_train, y_train = dataset
    bat_per_epo = 42
    for epoch in range(1, n_epochs + 1):
        print(f"Epoch {epoch}/{n_epochs}")
        for batch in range(bat_per_epo):
            # Get real samples
            ix = np.random.randint(0, X_train.shape[0], n_batch)
            X_real, labels_real = X_train[ix], y_train[ix]
            y_real = np.ones((n_batch, 1))
            # Update discriminator on real samples
            d_loss_real,_= d_model.train_on_batch([X_real, labels_real], y_real)
            # Generate fake samples
            X_fake, labels_fake = generate_fake_samples(g_model, latent_dim, n_batch)
            y_fake = np.zeros((n_batch, 1))
            # Update discriminator on fake samples
            d_loss_fake,_= d_model.train_on_batch([X_fake, labels_fake], y_fake)
            # Prepare points in latent space as input for the generator
            z_input = np.random.randn(n_batch * latent_dim)
            z_input = z_input.reshape(n_batch, latent_dim)
            labels_gan = np.random.randint(0, 7, n_batch).reshape(-1, 1)
            y_gan = np.ones((n_batch, 1))
            # Update the generator via the discriminator's error
            g_loss= gan_model.train_on_batch([z_input, labels_gan], y_gan)

            print(f"  Batch {batch + 1}/{bat_per_epo}, d_real_loss: {d_loss_real:.4f}, "
                  f"d_fake_loss: {d_loss_fake:.4f},g_loss: {g_loss:.4f}")

        # Save generated images every few epochs
        if epoch % 100 == 0:
          examples=10
          latent_dim=100
          generator=g_model
          latent_points = np.random.randn(examples * latent_dim)
          latent_points = latent_points.reshape(examples, latent_dim)
          labels = np.random.randint(0, 7, examples).reshape(-1, 1)
          X_fake = generator.predict([latent_points, labels])
          plt.figure(figsize=(10, 10))
          for i in range(examples):
            plt.subplot(1, examples, i + 1)
            plt.imshow(X_fake[i].reshape(48, 48), cmap='gray')
            plt.title('Label: %d' % labels[i])
            plt.axis('off')
          filename = save_dir + '/generated_plot_epoch_%04d.png' % (epoch)
          plt.savefig(filename)
          plt.close()



##Run the GAN

In [ ]:
def run_cgan():
    # Load FER2013 dataset
    dataset = load_fer2013_dataset()
    X_train, y_train = dataset
    # Create discriminator and generator models
    d_model = define_discriminator()
    g_model = define_generator(latent_dim=100)
    # Create GAN model
    gan_model = define_gan(g_model, d_model)
    # Train the GAN
    train_gan(g_model, d_model, gan_model, (X_train, y_train), latent_dim=100, n_epochs=10000, n_batch=128)
    # Save discriminator model
    save_model(d_model, 'discriminator_model.h5')  # Save discriminator model
    # Save generator model
    save_model(g_model, 'generator_model.h5')  # Save generator model
    # Save GAN model (including both generator and discriminator)
    save_model(gan_model, 'cgan_model.h5')  # Save CGAN model
run_cgan()


Streaming output truncated to the last 5000 lines.
  Batch 8/42, d_real_loss: 0.6407, d_fake_loss: 0.6709,g_loss: 0.9362
4/4 [==============================] - 0s 11ms/step
  Batch 9/42, d_real_loss: 0.6685, d_fake_loss: 0.6605,g_loss: 0.8320
4/4 [==============================] - 0s 9ms/step
  Batch 10/42, d_real_loss: 0.6694, d_fake_loss: 0.6495,g_loss: 1.0363
4/4 [==============================] - 0s 9ms/step
  Batch 11/42, d_real_loss: 0.6399, d_fake_loss: 0.6688,g_loss: 0.9771
4/4 [==============================] - 0s 8ms/step
  Batch 12/42, d_real_loss: 0.6311, d_fake_loss: 0.6539,g_loss: 0.9507
4/4 [==============================] - 0s 9ms/step
  Batch 13/42, d_real_loss: 0.6495, d_fake_loss: 0.6182,g_loss: 0.9107
4/4 [==============================] - 0s 8ms/step
  Batch 14/42, d_real_loss: 0.6373, d_fake_loss: 0.6984,g_loss: 0.9527
4/4 [==============================] - 0s 9ms/step
  Batch 15/42, d_real_loss: 0.6136, d_fake_loss: 0.6619,g_loss: 1.0176
4/4 [====================

<ipython-input-17-e06196bc7287>:47: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  plt.title('Label: %d' % labels[i])


Streaming output truncated to the last 5000 lines.
4/4 [==============================] - 0s 9ms/step
  Batch 33/42, d_real_loss: 0.6267, d_fake_loss: 0.6615,g_loss: 0.9565
4/4 [==============================] - 0s 9ms/step
  Batch 34/42, d_real_loss: 0.6799, d_fake_loss: 0.6328,g_loss: 0.8674
4/4 [==============================] - 0s 9ms/step
  Batch 35/42, d_real_loss: 0.6168, d_fake_loss: 0.6479,g_loss: 0.8937
4/4 [==============================] - 0s 9ms/step
  Batch 36/42, d_real_loss: 0.6432, d_fake_loss: 0.6590,g_loss: 0.9524
4/4 [==============================] - 0s 9ms/step
  Batch 37/42, d_real_loss: 0.6525, d_fake_loss: 0.6466,g_loss: 0.8582
4/4 [==============================] - 0s 9ms/step
  Batch 38/42, d_real_loss: 0.6447, d_fake_loss: 0.6269,g_loss: 0.9363
4/4 [==============================] - 0s 9ms/step
  Batch 39/42, d_real_loss: 0.6619, d_fake_loss: 0.7235,g_loss: 0.9058
4/4 [==============================] - 0s 9ms/step
  Batch 40/42, d_real_loss: 0.6413, d_fake_l

In [ ]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples, n_class):
 # generate points in the latent space
  x_input = randn(latent_dim * n_samples)
 # reshape into a batch of inputs for the network
  z_input = x_input.reshape(n_samples, latent_dim)
 # generate labels
  labels = asarray([n_class for _ in range(n_samples)])
  return [z_input, labels]

# create and save a plot of generated images
def save_plot(examples, n_examples):
    # Calculate the number of rows and columns for subplot arrangement
    n_rows = int(sqrt(n_examples))
    n_cols = int(sqrt(n_examples))

    # Plot images
    plt.figure(figsize=(8, 8))  # Adjust the figure size if needed
    for i in range(n_examples):
        # Define subplot
        plt.subplot(n_rows, n_cols, 1 + i)
        # Turn off axis
        plt.axis('off')
        # Plot raw pixel data
        plt.imshow(examples[i, :, :, 0], cmap='gray_r')

    # Show the plot
    plt.show()


# load model
model =load_model('generator_model.h5')
latent_dim = 100
n_examples = 100 # must be a square
n_class = 0 # sneaker
# generate images
latent_points, labels = generate_latent_points(latent_dim, n_examples, n_class)
# generate images
X  = model.predict([latent_points, labels])
# scale from [-1,1] to [0,1]
X = (X + 1) / 2.0
# plot the result
save_plot(X, n_examples)
